<a href="https://colab.research.google.com/github/gorzanskik-ai/intro-to-ann/blob/main/03_keras/04_overfitting_underfitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.datasets.imdb import get_word_index
from tensorflow.keras.utils import get_file
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [4]:
NUM_WORDS = 10000   # 10000 najczęściej pojawiających się słów
INDEX_FROM = 3

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=NUM_WORDS, index_from=INDEX_FROM)

In [5]:
word_to_idx = get_word_index()
word_to_idx = {k:(v + INDEX_FROM) for k, v in word_to_idx.items()}
word_to_idx['<PAD>'] = 0
word_to_idx['<START>'] = 1
word_to_idx['<UNK>'] = 2
word_to_idx['<UNUSED>'] = 3
idx_to_word = {v: k for k, v in word_to_idx.items()}
list(idx_to_word.items())[:10]
print(' '.join(idx_to_word[idx] for idx in train_data[0]))

<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wha

In [6]:
def multi_hot_sequences(sequences, dimension):
    results = np.zeros((len(sequences), dimension))
    for i, word_indices in enumerate(sequences):
        results[i, word_indices] = 1.0
    return results

train_data = multi_hot_sequences(train_data, dimension=NUM_WORDS)
test_data = multi_hot_sequences(test_data, dimension=NUM_WORDS)
train_data.shape

(25000, 10000)

In [7]:
#model bazowy
baseline_model = Sequential()
baseline_model.add(Dense(16, activation='relu', input_shape=(NUM_WORDS,)))
baseline_model.add(Dense(16, activation='relu'))
baseline_model.add(Dense(1, activation='sigmoid'))

baseline_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

baseline_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                160016    
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [8]:
baseline_history = baseline_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 [==============================] - 4s 60ms/step - loss: 0.5204 - accuracy: 0.7769 - binary_crossentropy: 0.5204 - val_loss: 0.3609 - val_accuracy: 0.8734 - val_binary_crossentropy: 0.3609
Epoch 2/20
49/49 [==============================] - 2s 42ms/step - loss: 0.2652 - accuracy: 0.9079 - binary_crossentropy: 0.2652 - val_loss: 0.2927 - val_accuracy: 0.8834 - val_binary_crossentropy: 0.2927
Epoch 3/20
49/49 [==============================] - 2s 43ms/step - loss: 0.1885 - accuracy: 0.9353 - binary_crossentropy: 0.1885 - val_loss: 0.2885 - val_accuracy: 0.8859 - val_binary_crossentropy: 0.2885
Epoch 4/20
49/49 [==============================] - 2s 42ms/step - loss: 0.1497 - accuracy: 0.9502 - binary_crossentropy: 0.1497 - val_loss: 0.3052 - val_accuracy: 0.8811 - val_binary_crossentropy: 0.3052
Epoch 5/20
49/49 [==============================] - 2s 43ms/step - loss: 0.1239 - accuracy: 0.9610 - binary_crossentropy: 0.1239 - val_loss: 0.3375 - val_accuracy: 0.8739 - val_bin

In [ ]:
#smaller model
smaller_model = Sequential()
smaller_model.add(Dense(4, activation='relu', input_shape=(NUM_WORDS, )))
smaller_model.add(Dense(4, activation='relu'))
smaller_model.add(Dense(1, activation='sigmoid'))
smaller_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

smaller_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 4)                 40004     
                                                                 
 dense_4 (Dense)             (None, 4)                 20        
                                                                 
 dense_5 (Dense)             (None, 1)                 5         
                                                                 
Total params: 40,029
Trainable params: 40,029
Non-trainable params: 0
_________________________________________________________________


In [ ]:
smaller_history = smaller_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels), verbose=0)

In [ ]:
#bigger model
bigger_model = Sequential()
bigger_model.add(Dense(512, activation='relu', input_shape=(NUM_WORDS,)))
bigger_model.add(Dense(512, activation='relu'))
bigger_model.add(Dense(1, activation='sigmoid'))

bigger_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

bigger_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 512)               5120512   
                                                                 
 dense_7 (Dense)             (None, 512)               262656    
                                                                 
 dense_8 (Dense)             (None, 1)                 513       
                                                                 
Total params: 5,383,681
Trainable params: 5,383,681
Non-trainable params: 0
_________________________________________________________________


In [ ]:
bigger_history = bigger_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 [==============================] - 16s 326ms/step - loss: 0.3461 - accuracy: 0.8496 - binary_crossentropy: 0.3461 - val_loss: 0.2989 - val_accuracy: 0.8769 - val_binary_crossentropy: 0.2989
Epoch 2/20
49/49 [==============================] - 15s 309ms/step - loss: 0.1454 - accuracy: 0.9476 - binary_crossentropy: 0.1454 - val_loss: 0.3310 - val_accuracy: 0.8727 - val_binary_crossentropy: 0.3310
Epoch 3/20
49/49 [==============================] - 15s 308ms/step - loss: 0.0490 - accuracy: 0.9853 - binary_crossentropy: 0.0490 - val_loss: 0.4629 - val_accuracy: 0.8648 - val_binary_crossentropy: 0.4629
Epoch 4/20
49/49 [==============================] - 15s 307ms/step - loss: 0.0084 - accuracy: 0.9984 - binary_crossentropy: 0.0084 - val_loss: 0.5744 - val_accuracy: 0.8696 - val_binary_crossentropy: 0.5744
Epoch 5/20
49/49 [==============================] - 15s 308ms/step - loss: 0.0013 - accuracy: 1.0000 - binary_crossentropy: 0.0013 - val_loss: 0.6837 - val_accuracy: 0.8701

In [ ]:
history = pd.DataFrame(baseline_history.history)
history['epochs'] = baseline_history.epoch
history.head()

,loss,accuracy,binary_crossentropy,val_loss,val_accuracy,val_binary_crossentropy,epochs
0,0.470381,0.81608,0.470381,0.331640,0.87644,0.331640,0
1,0.245009,0.91208,0.245009,0.282960,0.88688,0.282960,1
2,0.179268,0.93708,0.179268,0.290984,0.88480,0.290984,2
3,0.140950,0.95228,0.140950,0.314004,0.87872,0.314004,3
4,0.114322,0.96204,0.114322,0.346875,0.87388,0.346875,4


In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
for name, history in zip(['smaller', 'baseline', 'bigger'], [smaller_history, baseline_history, bigger_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()

In [7]:
from tensorflow.keras.regularizers import l2

l2_model = Sequential()
l2_model.add(Dense(16, kernel_regularizer=l2(0.001), activation='relu', input_shape=(NUM_WORDS,)))
l2_model.add(Dense(16, kernel_regularizer=l2(0.01), activation='relu'))
l2_model.add(Dense(1, activation='sigmoid'))

l2_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy', 'binary_crossentropy'])

l2_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                160016    
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [8]:
l2_model_history = l2_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 [==============================] - 5s 65ms/step - loss: 0.6551 - accuracy: 0.8109 - binary_crossentropy: 0.4744 - val_loss: 0.5026 - val_accuracy: 0.8785 - val_binary_crossentropy: 0.3355
Epoch 2/20
49/49 [==============================] - 3s 58ms/step - loss: 0.4177 - accuracy: 0.9088 - binary_crossentropy: 0.2574 - val_loss: 0.4410 - val_accuracy: 0.8866 - val_binary_crossentropy: 0.2894
Epoch 3/20
49/49 [==============================] - 2s 46ms/step - loss: 0.3486 - accuracy: 0.9306 - binary_crossentropy: 0.2038 - val_loss: 0.4208 - val_accuracy: 0.8865 - val_binary_crossentropy: 0.2832
Epoch 4/20
49/49 [==============================] - 2s 46ms/step - loss: 0.3118 - accuracy: 0.9392 - binary_crossentropy: 0.1799 - val_loss: 0.4149 - val_accuracy: 0.8827 - val_binary_crossentropy: 0.2892
Epoch 5/20
49/49 [==============================] - 2s 46ms/step - loss: 0.2865 - accuracy: 0.9463 - binary_crossentropy: 0.1649 - val_loss: 0.4087 - val_accuracy: 0.8813 - val_bin

In [12]:
import plotly.graph_objects as go

fig = go.Figure()
for name, history in zip(['baseline', 'l2'], [baseline_history, l2_model_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()

In [13]:
from tensorflow.keras.layers import Dropout

dropout_model = Sequential()
dropout_model.add(Dense(16, activation='relu', input_shape=(NUM_WORDS,)))
dropout_model.add(Dropout(0.5))
dropout_model.add(Dense(16, activation='relu'))
dropout_model.add(Dropout(0.5))
dropout_model.add(Dense(1, activation='sigmoid'))

dropout_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy', 'binary_crossentropy'])

dropout_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 16)                160016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_7 (Dense)             (None, 16)                272       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_8 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [14]:
dropout_history = dropout_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 [==============================] - 4s 71ms/step - loss: 0.6125 - accuracy: 0.6637 - binary_crossentropy: 0.6125 - val_loss: 0.4528 - val_accuracy: 0.8587 - val_binary_crossentropy: 0.4528
Epoch 2/20
49/49 [==============================] - 2s 44ms/step - loss: 0.4333 - accuracy: 0.8164 - binary_crossentropy: 0.4333 - val_loss: 0.3234 - val_accuracy: 0.8825 - val_binary_crossentropy: 0.3234
Epoch 3/20
49/49 [==============================] - 2s 45ms/step - loss: 0.3363 - accuracy: 0.8738 - binary_crossentropy: 0.3363 - val_loss: 0.2843 - val_accuracy: 0.8897 - val_binary_crossentropy: 0.2843
Epoch 4/20
49/49 [==============================] - 2s 49ms/step - loss: 0.2787 - accuracy: 0.8994 - binary_crossentropy: 0.2787 - val_loss: 0.2733 - val_accuracy: 0.8907 - val_binary_crossentropy: 0.2733
Epoch 5/20
49/49 [==============================] - 3s 58ms/step - loss: 0.2453 - accuracy: 0.9179 - binary_crossentropy: 0.2453 - val_loss: 0.2770 - val_accuracy: 0.8886 - val_bin

In [15]:
fig = go.Figure()
for name, history in zip(['baseline', 'dropout'], [baseline_history, dropout_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()

In [11]:
# dropout + regularizer
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dropout

dr_model = Sequential()
dr_model.add(Dense(16, kernel_regularizer=l2(0.001), activation='relu', input_shape=(NUM_WORDS,)))
dr_model.add(Dropout(0.2))
dr_model.add(Dense(16, kernel_regularizer=l2(0.01), activation='relu'))
dr_model.add(Dropout(0.2))
dr_model.add(Dense(1, activation='sigmoid'))

dr_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy', 'binary_crossentropy'])

dr_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 16)                160016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 16)                272       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_6 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [12]:
dr_history = dr_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 [==============================] - 4s 60ms/step - loss: 0.7132 - accuracy: 0.7426 - binary_crossentropy: 0.5550 - val_loss: 0.5448 - val_accuracy: 0.8685 - val_binary_crossentropy: 0.4023
Epoch 2/20
49/49 [==============================] - 2s 43ms/step - loss: 0.4991 - accuracy: 0.8643 - binary_crossentropy: 0.3623 - val_loss: 0.4472 - val_accuracy: 0.8824 - val_binary_crossentropy: 0.3161
Epoch 3/20
49/49 [==============================] - 2s 43ms/step - loss: 0.4167 - accuracy: 0.8985 - binary_crossentropy: 0.2901 - val_loss: 0.4166 - val_accuracy: 0.8842 - val_binary_crossentropy: 0.2953
Epoch 4/20
49/49 [==============================] - 2s 44ms/step - loss: 0.3670 - accuracy: 0.9135 - binary_crossentropy: 0.2492 - val_loss: 0.3958 - val_accuracy: 0.8864 - val_binary_crossentropy: 0.2819
Epoch 5/20
49/49 [==============================] - 2s 43ms/step - loss: 0.3301 - accuracy: 0.9250 - binary_crossentropy: 0.2195 - val_loss: 0.3884 - val_accuracy: 0.8847 - val_bin

In [15]:
import plotly.graph_objects as go

fig = go.Figure()
for name, history in zip(['baseline', 'dr'], [baseline_history, dr_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()